## A07- Árboles de decisión
### Paola Figueroa
### 6/11/2025

In [1]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt import gp_minimize
from sklearn.metrics import roc_auc_score
from scipy.stats import norm
from sklearn.tree import DecisionTreeRegressor

####  Tomar Advertising.csv y hacer una regresión linear como benchmark R2

In [2]:
data= pd.read_csv("/Users/paofigueroa/Documents/sem 5/Lab de aprendizaje estadístico/Advertising.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


In [3]:
x1=data.TV.values.reshape([-1,1])
x2=data.radio.values.reshape([-1,1])
x3=data.newspaper.values.reshape([-1,1])
y=data.sales

In [4]:
lr=LinearRegression()
X=np.hstack((x1,x2,x3))
lr.fit(X,y)
lr.intercept_, lr.coef_
B0= lr.intercept_
Beta1=lr.coef_
B0, Beta1

(2.9388893694594014, array([ 0.04576465,  0.18853002, -0.00103749]))

In [5]:
r2= lr.score(X,y)
r2

0.8972106381789522

#### Usar un decision tree y ajustar con dataset R2

In [6]:
decision_tree= DecisionTreeRegressor()
decision_tree.fit(X,y)
r2_dt= decision_tree.score(X,y)
r2_dt

1.0

#### Comparar R2 y poner observaciones

Podemos ver que el R2 usando linear regression es de 0.89, mientras que la de los árboles es de 1.

#### Repetir (1-3) con train-test split (random state=12)

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.6, random_state=42)
lr_tt= LinearRegression()
lr_tt.fit(X_train, y_train)
#r2 en test
r2_tt= lr_tt.score(X_test, y_test)
print("R2 en test: ",r2_tt)

#R2 en train
r2_tt_train= lr_tt.score(X_train, y_train)
print('R2 en train: ',r2_tt_train)


R2 en test:  0.8836868465767361
R2 en train:  0.8966046758114574


In [8]:
#Decision tree
decision_tree_tt= DecisionTreeRegressor()
decision_tree_tt.fit(X_train, y_train)
#r2 en test
r2_dt_tt= decision_tree_tt.score(X_test, y_test)
print("R2 en test (DT): ",r2_dt_tt)
#R2 en train
r2_dt_tt_train= decision_tree_tt.score(X_train, y_train)
print('R2 en train (DT): ',r2_dt_tt_train)

R2 en test (DT):  0.9246780280771802
R2 en train (DT):  1.0


#### Repetir (1-3) con Polynomial Features (degree=2)

In [17]:
X

array([[230.1,  37.8,  69.2],
       [ 44.5,  39.3,  45.1],
       [ 17.2,  45.9,  69.3],
       [151.5,  41.3,  58.5],
       [180.8,  10.8,  58.4],
       [  8.7,  48.9,  75. ],
       [ 57.5,  32.8,  23.5],
       [120.2,  19.6,  11.6],
       [  8.6,   2.1,   1. ],
       [199.8,   2.6,  21.2],
       [ 66.1,   5.8,  24.2],
       [214.7,  24. ,   4. ],
       [ 23.8,  35.1,  65.9],
       [ 97.5,   7.6,   7.2],
       [204.1,  32.9,  46. ],
       [195.4,  47.7,  52.9],
       [ 67.8,  36.6, 114. ],
       [281.4,  39.6,  55.8],
       [ 69.2,  20.5,  18.3],
       [147.3,  23.9,  19.1],
       [218.4,  27.7,  53.4],
       [237.4,   5.1,  23.5],
       [ 13.2,  15.9,  49.6],
       [228.3,  16.9,  26.2],
       [ 62.3,  12.6,  18.3],
       [262.9,   3.5,  19.5],
       [142.9,  29.3,  12.6],
       [240.1,  16.7,  22.9],
       [248.8,  27.1,  22.9],
       [ 70.6,  16. ,  40.8],
       [292.9,  28.3,  43.2],
       [112.9,  17.4,  38.6],
       [ 97.2,   1.5,  30. ],
       [26

In [16]:
X_poly

array([[ 230.1 ,   37.8 ,   69.2 , ..., 1428.84, 2615.76, 4788.64],
       [  44.5 ,   39.3 ,   45.1 , ..., 1544.49, 1772.43, 2034.01],
       [  17.2 ,   45.9 ,   69.3 , ..., 2106.81, 3180.87, 4802.49],
       ...,
       [ 177.  ,    9.3 ,    6.4 , ...,   86.49,   59.52,   40.96],
       [ 283.6 ,   42.  ,   66.2 , ..., 1764.  , 2780.4 , 4382.44],
       [ 232.1 ,    8.6 ,    8.7 , ...,   73.96,   74.82,   75.69]])

In [18]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.6, random_state=42)
model_poly = LinearRegression()
model_poly.fit(X_train, y_train)

#R2 test
r2_tt = model_poly.score(X_test, y_test)
print("R2 en test: ", r2_tt)

# R2 en train
r2_tt_train_poly = model_poly.score(X_train, y_train)
print('R2 en train: ', r2_tt_train_poly)

R2 en test:  0.975148160857558
R2 en train:  0.98607741214029


In [26]:
#Decision Tree
decision_tree_poly = DecisionTreeRegressor()
decision_tree_poly.fit(X_train, y_train)

# R2 en test
r2_dt_tt = decision_tree_poly.score(X_test, y_test)
print(f"R2 en test (Árbol con Poly): {r2_dt_tt}")

# 4. R2 en train
r2_dt_tt_train = decision_tree_poly.score(X_train, y_train)
print(f"R2 en train (Árbol con Poly): {r2_dt_tt_train}")

R2 en test (Árbol con Poly): 0.9557535907457941
R2 en train (Árbol con Poly): 1.0


Da diferente porque no son convexos